## 0. Paths and Config

In [ ]:
# Set SNT Paths
SNT_ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(SNT_ROOT_PATH, "code")
CONFIG_PATH    <- file.path(SNT_ROOT_PATH, "configuration")

REPORTING_NB_PATH <- file.path(SNT_ROOT_PATH, "pipelines/snt_dhis2_population_transformation/reporting")

In [ ]:
# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))
source(file.path(CODE_PATH, "snt_palettes.r"))

In [ ]:
required_packages <- c(
    "tidyverse", 
    "arrow", 
    "sf", 
    "reticulate"
    # "patchwork"
) 

# Execute function
install_and_load(required_packages)

In [ ]:
# Set environment to load openhexa.sdk from the right environment
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")

# Load openhexa.sdk
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

In [ ]:
# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

In [ ]:
# Configuration variables
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_POPULATION_TRANSFORMATION
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
COUNTRY_NAME <- config_json$SNT_CONFIG$COUNTRY_NAME
ADM_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

In [ ]:
    # --- Read data from SNT_metadata.json ---
    metadata_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_metadata.json"))},
     error = function(e) {
           msg <- paste0("Error while loading metadata", conditionMessage(e))  
            cat(msg)   
          stop(msg) 
    })

### Helper function(s)

In [ ]:
# print function
printdim <- function(df, name = deparse(substitute(df))) {
  cat("Dimensions of", name, ":", nrow(df), "rows x", ncol(df), "columns\n\n")
}

## 1. Import data

In [ ]:
population_data <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_population.parquet")) }, 
                  error = function(e) {
                      msg <- paste0(COUNTRY_NAME , " Population data is not available in dataset : " , dataset_name, " last version.")
                      log_msg(msg, "warning")
                      population_data <- NULL
                      })

printdim(population_data)

In [ ]:
shapes_dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

shapes_data <- tryCatch({ get_latest_dataset_file_in_memory(shapes_dataset_name, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {                      
                      msg <- paste0(COUNTRY_NAME , " Shapes data is not available in dataset : " , shapes_dataset_name, " last version.")
                      log_msg(msg, "warning")
                      shapes_data <- NULL
                      })

printdim(shapes_data)

# 3. Carte des populations par district sanitaire (DS)

In [ ]:
# Code from previous version of the notebook
# Uses continuos scale for population

# Run if population_data is available
if (!is.null(population_data) & !is.null(shapes_data)) {
    # Join population to spatial shapes
    map_data <- shapes_data %>%
      left_join(population_data, by = "ADM2_ID")
    
    # Plot population per district (DS)
    plot <- ggplot(map_data) +
      geom_sf(aes(fill = POPULATION), color = "white", size = 0.2) +
      scale_fill_viridis_c(option = "C", name = "Population") +
      labs(
        title = "Population totale par district sanitaire (DS)",
        subtitle = "Données DHIS2",
        caption = "Source: NMDR / DHIS2"
      ) +
      theme_minimal(base_size = 14) 

    print(plot)

} else {
    print("Population or shapes data not available.")
}


## ⚠️ 3.2. Carte de la Population Désagrégée (spécifique au pays)
Le code suivant est spécifique à chaque pays et repose sur une population désagrégée. 

### 🇳🇪 NER specific code 
Made ad hoc to allow comparison with data from other or previous analyses. Namely:
* only year 2022 to 2024
* specific palette (yellowish to brick red)
* specific intervals
* looks at **disaggregated** population <- this is sometimes contry-specific!

In [ ]:
if (COUNTRY_CODE == "NER") {
    print("🇳🇪 Executing NER specific code ... ")

    # --- Filter data to keep only 2022-2024 ... ---
    years_to_keep <- 2022:2024
    population_data_filtered <- population_data |> filter(YEAR %in% years_to_keep)

    # # --- Read data from SNT_metadata.json ---
    # metadata_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_metadata.json"))},
    #  error = function(e) {
    #        msg <- paste0("Error while loading metadata", conditionMessage(e))  
    #         cat(msg)   
    #       stop(msg) 
    # })

    # --- Assign population breaks from metadata ---
    value_breaks_tot <- jsonlite::fromJSON(metadata_json$POPULATION_TOTAL$SCALE)
    value_breaks_u5 <- jsonlite::fromJSON(metadata_json$POPULATION_U5$SCALE)
    value_breaks_fe <- jsonlite::fromJSON(metadata_json$POPULATION_PREGNANT$SCALE)

    # --- Define function to create dyanic labels based on breaks for pop category ---
    create_dynamic_labels <- function(breaks) {
    fmt <- function(x) {
        format(x / 1000, big.mark = "'", scientific = FALSE, trim = TRUE)
        }
  
    labels <- c(
        paste0("< ", fmt(breaks[1]), "k"),                          # First label
        paste0(fmt(breaks[-length(breaks)]), " - ", fmt(breaks[-1]), "k"), # Middle
        paste0("> ", fmt(breaks[length(breaks)]), "k")               # Last label
    ) 
    return(labels)
    }

    # --- Create dynamic labels based on breaks ---
    labels_tot <- create_dynamic_labels(value_breaks_tot)
    labels_u5 <- create_dynamic_labels(value_breaks_u5)
    labels_fe <- create_dynamic_labels(value_breaks_fe)

}

In [ ]:
NER_palette_population <- c(
  "1" = "#fae6db",
  "2" = "#f1b195",
  "3" = "#ea7354",
  "4" = "#cc3f32",
  "5" = "#972620"
)


### Population Totales

In [ ]:
if (COUNTRY_CODE == "NER") {

  # IMPORTNAT: palette vector MUST be RENAMED with the (dynamic) descriptive labels
names(NER_palette_population) <- labels_tot

plot <- population_data_filtered %>%
  mutate(
    CATEGORY_POPULATION = cut(
      POPULATION,
      breaks = c(0, value_breaks_tot, Inf),
      labels = labels_tot, 
      right = TRUE,
      include.lowest = TRUE
    )
  ) %>% 
  left_join(shapes_data, 
            by = join_by(ADM1_NAME, ADM1_ID, ADM2_NAME, ADM2_ID)) %>% 
  ggplot() +
  geom_sf(aes(geometry = geometry,
              fill = CATEGORY_POPULATION),
          color = "black",
          linewidth = 0.25, 
          show.legend = TRUE
  ) +
  labs(
    title = "Population totale par district sanitaire (DS)",
    subtitle = "Source: NMDR / DHIS2",
    fill = "Population Totale:"
  ) +
  scale_fill_manual(
    values = NER_palette_population,   
    limits = labels_tot, 
    drop = FALSE 
  ) +
  facet_wrap(~YEAR, ncol = 3) +
  theme_void() +
  theme(
    plot.title = element_text(face = "bold"),
    plot.subtitle = element_text(margin = margin(5, 0, 20, 0)),
    legend.position = "bottom",
    legend.title.position = "top",
    legend.title = element_text(face = "bold"),
    strip.text = element_text(face = "bold"),
    legend.key.height = unit(0.5, "line"),
    legend.margin = margin(20, 0, 0, 0)
  )

print(plot)

# Export to see better in high resolution
ggsave(
  filename = file.path(REPORTING_NB_PATH, "outputs", "figures", paste0(COUNTRY_CODE, "_choropleth_population_transformed_total.png")),
  create.dir = TRUE,
  units = "cm",
  width = 21,
  height = 15,
  dpi = 300
)
}


### Population Femmes Enceintes (FE)

In [ ]:
if (COUNTRY_CODE == "NER") {

names(NER_palette_population) <- labels_fe

plot <- population_data_filtered %>%
  mutate(
    CATEGORY_POPULATION = cut(
      POPULATION_FE,
      breaks = c(0, value_breaks_fe, Inf),
      labels = labels_fe, 
      right = TRUE,
      include.lowest = TRUE
    )
  ) %>% 
  left_join(shapes_data, 
            by = join_by(ADM1_NAME, ADM1_ID, ADM2_NAME, ADM2_ID)) %>% 
  ggplot() +
  geom_sf(aes(geometry = geometry,
              fill = CATEGORY_POPULATION),
          color = "black",
          linewidth = 0.25, 
          show.legend = TRUE
  ) +
  labs(
    title = "Population des femmes enceintes par district sanitaire (DS)",
    subtitle = "Source: NMDR / DHIS2",
    fill = "Population Femmes Enceintes:"
  ) +
  scale_fill_manual(
    values = NER_palette_population,   
    limits = labels_fe, 
    drop = FALSE # Prevents dropping empty levels from legend
  ) +
  facet_wrap(~YEAR, ncol = 3) +
  theme_void() +
  theme(
    plot.title = element_text(face = "bold"),
    plot.subtitle = element_text(margin = margin(5, 0, 20, 0)),
    legend.position = "bottom",
    legend.title = element_text(face = "bold"),
    legend.title.position = "top",
    strip.text = element_text(face = "bold"),
    legend.key.height = unit(0.5, "line"),
    legend.margin = margin(20, 0, 0, 0)
  )

print(plot)

# Export to see better in high resolution
ggsave(
  filename = file.path(REPORTING_NB_PATH, "outputs", "figures", paste0(COUNTRY_CODE, "_choropleth_population_transformed_fe.png")),
  create.dir = TRUE,
  units = "cm",
  width = 21,   
  height = 15,
  dpi = 300
)

}


### Population Enfants moins de 5 ans (U5)

In [ ]:
if (COUNTRY_CODE == "NER") {

names(NER_palette_population) <- labels_u5

plot <- population_data_filtered %>%
  mutate(
    CATEGORY_POPULATION = cut(
      POPULATION_U5,
      breaks = c(0, value_breaks_u5, Inf),
      labels = labels_u5, 
      right = TRUE,
      include.lowest = TRUE
    )
  ) %>% 
  left_join(shapes_data, 
            by = join_by(ADM1_NAME, ADM1_ID, ADM2_NAME, ADM2_ID)) %>% 
  ggplot() +
  geom_sf(aes(geometry = geometry,
              fill = CATEGORY_POPULATION),
          color = "black",
          linewidth = 0.25, 
          show.legend = TRUE
  ) +
  labs(
    title = "Population des enfants de moins de 5 ans par district sanitaire (DS)",
    subtitle = "Source: NMDR / DHIS2",
    fill = "Population Enfants de moins de 5 ans:"
  ) +
  scale_fill_manual(
    values = NER_palette_population,   
    limits = labels_u5, 
    drop = FALSE 
  ) +
  facet_wrap(~YEAR, ncol = 3) +
  theme_void() +
  theme(
    plot.title = element_text(face = "bold"),
    plot.subtitle = element_text(margin = margin(5, 0, 20, 0)),
    legend.position = "bottom",
    legend.title = element_text(face = "bold"),
    legend.title.position = "top",
    strip.text = element_text(face = "bold"),
    legend.key.height = unit(0.5, "line"),
    legend.margin = margin(20, 0, 0, 0)
  )

print(plot)

# Export PNG
ggsave(
  filename = file.path(REPORTING_NB_PATH, "outputs", "figures", paste0(COUNTRY_CODE, "_choropleth_population_transformed_u5.png")),
  create.dir = TRUE,
  units = "cm",
  width = 21,   
  height = 15,
  dpi = 300
)

}